# Unicorn Carousel Quote Pipeline Example

<p align="center">
  <img src="unicorn_bill.png" />
</p>


## Introduction
The goal if this sample notebook is to demonstrate a more complex pipeline using all three types of components and some simple data passing. This pipeline generates a fictionl quote for a magical unicorn carousel based on the radius of the carousel,the cost of materials, and the local state taxes. Our quote adjusts based on the size of the required carousel. We use two Python components and an R component. The final generate_quote component is written in R. 

## Intial Import Step

In [52]:
import kfp
from kfp import dsl
from kfp import compiler
from kfp.client import Client


client = kfp.Client()
print(client.list_experiments())

{'experiments': [{'created_at': datetime.datetime(2024, 2, 17, 17, 59, 4, tzinfo=tzlocal()),
                  'description': 'my first Kubeflow experiment! ',
                  'display_name': 'hello_kubeflow',
                  'experiment_id': 'c9e56f11-3433-4cef-a1a4-d3fb20c9b26e',
                  'namespace': 'christensenc3526',
                  'storage_state': 'AVAILABLE'},
                 {'created_at': datetime.datetime(2024, 2, 18, 16, 5, 37, tzinfo=tzlocal()),
                  'description': None,
                  'display_name': 'new_experiment',
                  'experiment_id': 'bc7afbde-6c81-41e5-b7ef-ef81545c36e7',
                  'namespace': 'christensenc3526',
                  'storage_state': 'AVAILABLE'},
                 {'created_at': datetime.datetime(2024, 2, 26, 4, 33, 34, tzinfo=tzlocal()),
                  'description': None,
                  'display_name': 'Default',
                  'experiment_id': '7671697d-de7a-47a8-975a-27b0b89575ee',
  

## Containerized Python Component Step

In [53]:
@dsl.component(target_image='chasechristensen/material_cost_component:v1')
def calculate_material_cost(radius: float, cost_per_square_unit: float) -> float:
    """Calculates the cost of materials needed to cover a circle's area."""
    area = calculate_area(radius)
    total_cost = area * cost_per_square_unit
    print(f"The total cost to cover the area of a circle with radius {radius} is: ${total_cost:.2f}")
    return total_cost

## Lightweight Python Component Step

In [55]:
@dsl.component
def get_state_tax_rate(state: str) -> float:
    """Returns the tax rate for a given state or raises an error if the state is invalid."""
    # Dictionary of state tax rates (simplified and fictional rates for illustration)
    state_tax_rates = {
        "AL": 0.04, "AK": 0.0, "AZ": 0.056, "AR": 0.065, "CA": 0.075,
        "CO": 0.029, "CT": 0.0635, "DE": 0.0, "FL": 0.06, "GA": 0.04,
        "HI": 0.04, "ID": 0.06, "IL": 0.0625, "IN": 0.07, "IA": 0.06,
        "KS": 0.065, "KY": 0.06, "LA": 0.0445, "ME": 0.055, "MD": 0.06,
        "MA": 0.0625, "MI": 0.06, "MN": 0.06875, "MS": 0.07, "MO": 0.04225,
        "MT": 0.0, "NE": 0.055, "NV": 0.0685, "NH": 0.0, "NJ": 0.06625,
        "NM": 0.05125, "NY": 0.04, "NC": 0.0475, "ND": 0.05, "OH": 0.0575,
        "OK": 0.045, "OR": 0.0, "PA": 0.06, "RI": 0.07, "SC": 0.06,
        "SD": 0.045, "TN": 0.07, "TX": 0.0625, "UT": 0.0485, "VT": 0.06,
        "VA": 0.053, "WA": 0.065, "WV": 0.06, "WI": 0.05, "WY": 0.04
    }

    # Check if the state is valid and return the tax rate
    if state.upper() in state_tax_rates:
        return state_tax_rates[state.upper()]
    else:
        # If the state is not valid, raise an error and exit
        raise ValueError(f"Invalid state: {state}. Please provide a valid US state abbreviation.")


## Containerized Component Step

In [56]:
@dsl.container_component
def generate_quote(total_cost: float, state_tax: float, quote: dsl.OutputPath(str)):
    """output a quote"""
    return dsl.ContainerSpec(
        image='chasechristensen/generate_quote:v1',
        command=[
            "Rscript","/usr/local/bin/generate_quote.r"
        ],
        args=[total_cost,state_tax,quote])

## Pipeline Declaration

In [57]:
@dsl.pipeline
def unicorn_carousel_construction_quote(carou_radius: float, mat_costs: float,st: str) -> str:
    task1 = calculate_material_cost(radius=carou_radius, cost_per_square_unit=mat_costs)
    task2 = get_state_tax_rate(state=st)
    task3= generate_quote(total_cost=task1.output,state_tax=task2.output)
    return task3.output


## Compiling the Pipeline

In [58]:
compiler.Compiler().compile(unicorn_carousel_construction_quote, 'pipeline.yml')

## Pipeline Submission

In [59]:
run = client.create_run_from_pipeline_package(
    'pipeline.yml',
    arguments={
        'carou_radius': 40.0,
        'mat_costs':50.35,
        'st': "AZ"
    },
)